# Stage 2 Usage Examples (LangChain Reasoning & Router)

This notebook demonstrates how to use the Stage 2 chains to query the synthetic payments dataset and obtain concise summaries.

- Routed Q&A via `router_chain.ask()`
- Direct code generation and execution via `query_chain`
- Summarization via `summary_chain`
- Optional: enabling LLM with `OPENAI_API_KEY` (otherwise heuristic fallback is used)


In [2]:
# optional: load .env for local development
#%pip install -q python-dotenv
from dotenv import load_dotenv
load_dotenv()  # loads variables from .env into the process env


False

In [ ]:
# Setup imports and paths
import os, sys
from pathlib import Path
import pandas as pd

# Ensure project root on path
PROJECT_ROOT = str(Path.cwd().parent)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print('Project root:', PROJECT_ROOT)
print('OPENAI_API_KEY set:', bool(os.getenv('OPENAI_API_KEY')))

DATA_PATH = str(Path(PROJECT_ROOT) / 'data' / 'payments.csv')
print('Data path:', DATA_PATH)


In [ ]:
# Load dataset preview
df = pd.read_csv(DATA_PATH, parse_dates=['timestamp'])
print(f"Rows: {len(df):,} | Columns: {len(df.columns)}")
df.head(3)


In [ ]:
# Example 1: Routed data Q&A
from src.chains.router_chain import ask

q1 = "Which merchants had the highest total revenue last month?"
resp1 = ask(q1)
print('Route:', resp1['route'])
print('Answer:', resp1['answer'])

# Convert table (list of dicts) back into a DataFrame for display
if resp1.get('table'):
    pd.DataFrame(resp1['table']).head(10)


In [ ]:
# Example 2: Another routed question
q2 = "Which country has the highest average transaction amount?"
resp2 = ask(q2)
print('Route:', resp2['route'])
print('Answer:', resp2['answer'])

if resp2.get('table'):
    pd.DataFrame(resp2['table']).head(10)


In [ ]:
# Example 3: Direct query chain usage
from src.chains.query_chain import run as run_query

q3 = "Top 10 merchants by total revenue"
res3 = run_query(q3)
print('Keys:', list(res3.keys()))

# Reconstruct DataFrame from 'split' orient
split = res3['table']
df3 = pd.DataFrame(split['data'], columns=split['columns'])
df3.head(10)


In [ ]:
# Example 4: Summarize a result explicitly
from src.chains.summary_chain import SummaryChain

summary = SummaryChain().run(q3, df3)
print(summary)


In [ ]:
# Example 5 (optional): Async ask()
# If the notebook environment has an event loop, fallback to sync.
try:
    import asyncio
    from src.chains.router_chain import ask_async
    
    def run_async_example():
        q = "What was the total payment volume last week?"
        try:
            resp = asyncio.run(ask_async(q))
        except RuntimeError:
            # Fallback if event loop is already running
            resp = ask(q)
        print('Route:', resp['route'])
        print('Answer:', resp['answer'])
        return resp
    
    _ = run_async_example()
except Exception as e:
    print('Async example skipped:', e)
